In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [2]:
# Variables iniciales
api_key = "insertar_token_de_google"
tipo_contrato = 'Arriendo' # venta, arriendo o arriendo_temporal
tipo_inmueble = 'Departamentos' # dpto, casa u oficina.
ubicacion_inmueble = 'las condes' # la comuna de la búsqueda
monto_minimo = 500000 # monto mínimo de la búsqueda
monto_maximo = 1300000 # monto máximo de la búsqueda
cant_paginas = 2 # número de páginas a recorrer
radio_busqueda = '300' # radio (en metros) de búsqueda de lugares cercanos
busqueda_rubros = ['restaurante', 'supermercado'] # rubro de lugares cercanos

In [3]:
driver = webdriver.Edge()
driver.get('https://www.portalinmobiliario.com/')

In [5]:
# Función para cerrar el banner de cookies
def cerrar_banner_cookies():
    try:
        # Verificar si el elemento existe y es visible en el DOM
        cookie_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "newCookieDisclaimerButton"))
        )
        
        # Verificar si el botón es clickeable
        if cookie_button.is_displayed():
            cookie_button.click()
            print("Banner de cookies cerrado")
        else:
            print("El banner de cookies no está visible")
    except TimeoutException:
        print("No se encontró el banner de cookies")
    except Exception as e:
        print(f"Error inesperado al intentar cerrar el banner de cookies: {e}")


In [6]:
try:
    cerrar_banner_cookies()
    wait = WebDriverWait(driver, 5)

    # Filtrar según botón de tipo contrato
    contrato_button = wait.until(EC.element_to_be_clickable((By.ID, ":R2l5r:-trigger")))
    contrato_button.click() 
    print("Tipo de contrato")
    tipo_contrato_opcion = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[text()='{tipo_contrato}']")))
    tipo_contrato_opcion.click()
    print(f"Seleccionado: {tipo_contrato}")

    # Filtrar según botón de tipo inmueble
    inmueble_button = wait.until(EC.element_to_be_clickable((By.ID, ":R4l5r:-trigger")))
    inmueble_button.click()  # Hacer clic para abrir el dropdown de inmuebles
    print(f"Tipo de inmueble")
    tipo_inmueble_opcion = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[text()='{tipo_inmueble}']")))
    tipo_inmueble_opcion.click()  
    print(f"Seleccionado: {tipo_inmueble}")

    # Ingresar comuna de búsqueda
    comuna_input = wait.until(EC.presence_of_element_located((By.ID, ":Rml5r:")))
    comuna_input.send_keys(ubicacion_inmueble)
    print(f"Comuna ingresada: {ubicacion_inmueble}")

    # Usar la primera recomendación
    primera_recomendacion = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'andes-list__item-action')]")))
    texto_recomendacion = primera_recomendacion.text
    print("Primera recomendación seleccionada")
    print(f"Texto de la primera recomendación: {texto_recomendacion}")
    primera_recomendacion.click()
  
    # Buscar
    buscar_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']")))
    buscar_button.click()
    print("Búsqueda realizada")
    time.sleep(2)

except Exception as e:
    print(f"Error ocurrió: {e}")


Banner de cookies cerrado
Tipo de contrato
Seleccionado: Arriendo
Tipo de inmueble
Seleccionado: Departamentos
Comuna ingresada: las condes
Primera recomendación seleccionada
Texto de la primera recomendación: Las Condes, RM (Metropolitana)
Búsqueda realizada


In [7]:
try:
    cerrar_banner_cookies()
    wait = WebDriverWait(driver, 2)
    # Esperar y localizar el campo de "Monto mínimo" por su data-testid
    monto_minimo_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="Minimum-price"]')))
    
    # Hacer clic para activar el campo, limpiarlo e ingresar el valor
    monto_minimo_input.click()  
    monto_minimo_input.clear()  # Limpiar el campo antes de ingresar un valor
    monto_minimo_input.send_keys(str(monto_minimo))  # Ingresar monto mínimo
    print(f"Monto mínimo ingresado: {monto_minimo}")

    # Esperar y localizar el campo de "Monto mínimo" por su data-testid
    monto_maximo_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="Maximum-price"]')))
    
    # Hacer clic para activar el campo, limpiarlo e ingresar el valor
    monto_maximo_input.click()  
    monto_maximo_input.clear()  # Limpiar el campo antes de ingresar un valor
    monto_maximo_input.send_keys(str(monto_maximo))  # Ingresar monto mínimo
    print(f"Monto maximo ingresado: {monto_maximo}")
    
    # Esperar y localizar el botón de "Aplicar" por su data-testid
    aplicar_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="submit-price"]')))
    # Hacer clic en el botón para aplicar el filtro
    aplicar_button.click()
    print("Filtro aplicado con éxito")

except Exception as e:
    print(f"Error ocurrió: {e}")

No se encontró el banner de cookies
Monto mínimo ingresado: 500000
Monto maximo ingresado: 1300000
Filtro aplicado con éxito


In [8]:
def extraer_url():
    try:
        datos_departamentos = []  # Lista local para almacenar los datos
        pagina_actual = 1  # Iniciar contador de páginas

        while pagina_actual <= cant_paginas:
            print(f"Extrayendo datos de la página {pagina_actual}...")
            
            # Extraer los enlaces de los departamentos
            departamentos = wait.until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//div[@class='poly-card__content']//h2[@class='poly-box poly-component__title']//a")
                )
            )
            for departamento in departamentos:
                enlace = departamento.get_attribute("href")
                datos_departamentos.append(enlace)  # Añadir el enlace a la lista

            # Navegación pagina siguiente
            try:
                siguiente_pagina = wait.until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//a[contains(@class, 'andes-pagination__link') and span[contains(text(), 'Siguiente')]]")
                    )
                )
                siguiente_pagina.click()  # Hacer clic en el botón "Siguiente"
                time.sleep(5)  # Esperar que cargue la siguiente página
                pagina_actual += 1  # Incrementar el contador de páginas
            except TimeoutException:
                print("Error: No se encontró el botón de siguiente página o la página está completamente cargada.")
                break  # Terminar si no hay más páginas
            except NoSuchElementException:
                print("Error: No se encontró el botón de siguiente página.")
                break 
            except Exception as e:
                print(f"Error: {e}")
                break  # Terminar si ocurrió otro error

        return datos_departamentos

    except TimeoutException:
        print("Error: No se encontraron los elementos dentro del tiempo de espera.")
        return []
    except Exception as e:
        print(f"Error al extraer información: {e}")
        return []

# Llamar la función para extraer las URLs
datos_departamentos = extraer_url()
print("Datos extraídos:", datos_departamentos)

Extrayendo datos de la página 1...
Extrayendo datos de la página 2...
Datos extraídos: ['https://portalinmobiliario.com/MLC-1555797337-spot-nueva-kennedy-_JM#polycard_client=search-nordic&position=1&search_layout=grid&type=item&tracking_id=4481f98d-6049-4444-b8e3-38027d49f436', 'https://portalinmobiliario.com/MLC-1525161537-vespucio-switch-_JM#polycard_client=search-nordic&position=2&search_layout=grid&type=item&tracking_id=4481f98d-6049-4444-b8e3-38027d49f436', 'https://portalinmobiliario.com/MLC-2648931672-augusto-leguia-_JM#polycard_client=search-nordic&position=3&search_layout=grid&type=item&tracking_id=4481f98d-6049-4444-b8e3-38027d49f436', 'https://portalinmobiliario.com/MLC-1408634227-somma-asturias-edificio-multifamily-_JM#polycard_client=search-nordic&position=4&search_layout=grid&type=item&tracking_id=4481f98d-6049-4444-b8e3-38027d49f436', 'https://portalinmobiliario.com/MLC-2814352114-dpto-amoblado-remodelado-2d2b-los-militares-cerca-del-metro-_JM#polycard_client=search-nord

In [9]:
# Crear un diccionario para almacenar las URLs, títulos, monedas y valores
dict_info = {}

# Iterar sobre las primeras tres URLs en datos_departamentos
for href in datos_departamentos: 
    try:
        driver.get(href)  # Navegar por cada URL
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ui-pdp-title")))  # Esperar que cargue el título

        # Extraer el título
        titulo = driver.find_element(By.CLASS_NAME, "ui-pdp-title").text

        # Extraer la dirección
        try:
            location_element = driver.find_element(By.CSS_SELECTOR, "div.ui-vip-location__subtitle p")
            direccion = location_element.text
        except Exception as e:
            direccion = "Error al obtener dirección"
            print(f"Error al procesar dirección: {e}")

        # Extraer el precio
        try:
            contenedor_precio = driver.find_element(By.CLASS_NAME, "andes-money-amount")
            meta_precio = contenedor_precio.find_element(By.XPATH, ".//meta[@itemprop='price']")
            valor = meta_precio.get_attribute("content") 
            simbolo_moneda = contenedor_precio.find_element(By.CLASS_NAME, "andes-money-amount__currency-symbol").text
            moneda = "UF" if simbolo_moneda == "UF" else "$"        # Determinar la moneda desde el elemento visible

        except Exception as e:
            moneda = "Error"
            valor = "Error"
            print(f"Error al procesar precio en {href}: {e}")

        # Guardar información en el diccionario
        dict_info[href] = {"titulo": titulo, "direccion": direccion, "moneda": moneda, "valor": valor}

    except Exception as e:
        
        print(f"Error al procesar {href}: {e}")
        dict_info[href] = {"titulo": "Error al obtener título", "direccion": "Error", "moneda": "Error", "valor": "Error"}

# Crear un DataFrame con los datos extraídos
df = pd.DataFrame.from_dict(dict_info, orient='index').reset_index()
df.columns = ['pag', 'titulo', 'direccion', 'moneda', 'valor']

# Mostrar el DataFrame
print("\nDatos extraídos desde portalinmobiliario")
df


Datos extraídos desde portalinmobiliario


,pag,titulo,direccion,moneda,valor
0,https://portalinmobiliario.com/MLC-1555797337-...,Spot Nueva Kennedy,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",UF,19.64
1,https://portalinmobiliario.com/MLC-1525161537-...,Vespucio Switch,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",$,879178
2,https://portalinmobiliario.com/MLC-2648931672-...,Augusto Leguía,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",$,815000
3,https://portalinmobiliario.com/MLC-1408634227-...,Somma Asturias - Edificio Multifamily,"Asturias 77, Metro Escuela Militar, Las Condes...",$,830000
4,https://portalinmobiliario.com/MLC-2814352114-...,Dpto Amoblado Remodelado 2d+2b Los Militares C...,"Los Militares 4717, Metro Escuela Militar, Las...",UF,27
...,...,...,...,...,...
91,https://portalinmobiliario.com/MLC-2814795702-...,Cristobal Colón,"Cristobal Colón, Rotonda Atenas, Las Condes, R...",$,1100000
92,https://portalinmobiliario.com/MLC-2814865334-...,Escuela Militar,"Escuela Militar, Metro Escuela Militar, Las Co...",UF,21
93,https://portalinmobiliario.com/MLC-2814795742-...,Metro Escuela Militar,"Metro Escuela Militar, Metro Escuela Militar, ...",$,680000
94,https://portalinmobiliario.com/MLC-2806714528-...,Departamento En Arriendo De 1 Dorm. En Las Condes,"La Cabaña Av. Las Condes, Mall Sport, Las Cond...",$,590000


In [10]:
# Cerrar el navegador al finalizar
driver.quit()